In [17]:
"""Sentiment Analysis, Naive Bayes
Group members: Michael (Zeyu) Li (zl310) and Chengyang Zhou (cz169)"""

# referred to this https://github.com/DrManishSharma/NLP/blob/master/SentiAnalysis.ipynb
import matplotlib.pyplot as plt

import numpy as np
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


from nltk.tokenize import RegexpTokenizer

import pandas as pd

dataset = df = pd.read_csv('./IMDBDataset.csv')
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english', ngram_range=(1, 1), tokenizer=token.tokenize)
text_counts = cv.fit_transform(dataset['review'])

X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['sentiment'], test_size=0.3, random_state=5)
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)
print(MNB.score(X_test, Y_test))

/home/zl310/anaconda3/envs/michael_ece684/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.8568666666666667
